In [2]:
import os
import pickle
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import MSTL, HoltWinters
from tqdm.autonotebook import tqdm

# this makes it so that the outputs of the predict methods have the id as a column 
# instead of as the index
os.environ['NIXTLA_ID_AS_COL'] = '1'

# = = = = = =
# own stuff
# = = = = = =
os.chdir('../')
import data_prepro as data_prepro  # noqa: E402
import model_train as model_train # noqa: E402
import model_fcast as model_fcast # noqa: E402
import model_eval as model_eval # noqa: E402

c:\Users\ytl_c\miniconda3\Lib\site-packages\statsforecast\core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
os.environ["LOKY_MAX_CPU_COUNT"] = "1"  # Replace "4" with the desired number of cores

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
fcast_hor = [36, 40, 44, 60, 64, 68] # in hours

# = = = = = = = = = = = = = 
# >>> generate prediction timestamps based on t0 = following thursday 00:00 dynamically
# = = = = = = = = = = = = = 

# get current date and set time to 00:00
t_now = pd.Timestamp.now(tz='CET').replace(hour=0, minute=0, second=0, microsecond=0)
t_now = '2024-01-24'
t_now = pd.Timestamp(t_now, tz='CET')

# get thursday and wednesday
days_left = 3 - t_now.weekday() # get days left till thursday
t_thursday = t_now + pd.Timedelta(days=days_left)
t_wednesday = t_thursday - pd.Timedelta(days=1)
t_sunday = t_thursday + pd.Timedelta(days=3)
print(f"wed = {t_wednesday}")
print(f"thu = {t_thursday}")
print(f"sun = {t_sunday}")

# t_monday_next = t_thursday + pd.Timedelta(days=4)
t_wednesday_next = t_wednesday + pd.Timedelta(days=7)
# print(f"next mon = {t_monday_next}")
print(f"next wed = {t_wednesday_next}")

# get required submission horizons
# based on fcast horizons generate timestamps from t0
subm_timestamps = []
for fcast in fcast_hor:
    subm_timestamps.append((t_thursday + pd.Timedelta(hours=fcast)))
print(subm_timestamps)

wed = 2024-01-24 00:00:00+01:00
thu = 2024-01-25 00:00:00+01:00
sun = 2024-01-28 00:00:00+01:00
next wed = 2024-01-31 00:00:00+01:00
[Timestamp('2024-01-26 12:00:00+0100', tz='CET'), Timestamp('2024-01-26 16:00:00+0100', tz='CET'), Timestamp('2024-01-26 20:00:00+0100', tz='CET'), Timestamp('2024-01-27 12:00:00+0100', tz='CET'), Timestamp('2024-01-27 16:00:00+0100', tz='CET'), Timestamp('2024-01-27 20:00:00+0100', tz='CET')]


## Test

### statsmodels & sktime MSTL (BAD)

In [4]:
# mstl = MSTL(df_energy_small['gesamt'], periods=[24, 24*7], iterate=1, 
#             stl_kwargs={"seasonal_deg": 0,
#                         "inner_iter": 2,
#                         "outer_iter": 0})

# res = mstl.fit()

# plt.rc("figure", figsize=(15, 10))
# res.plot()
# plt.tight_layout()
# plt.show()

In [5]:
# # from sktime.datasets import load_airline
# from sktime.forecasting.statsforecast import StatsForecastMSTL

# y = df_energy_small['gesamt']
# model = StatsForecastMSTL(season_length=[24, 24*7], pred_int_kwargs={'n_windows':2, 'h':12})
# fitted_model = model.fit(y=y) 

In [6]:
# y_pred = fitted_model.predict(fh=np.arange(1,69)) 

# plt.figure(figsize=(15, 5))
# plt.plot(y_pred)
# plt.show()

In [7]:
# y_pred = fitted_model.predict_interval(fh=np.arange(1,69), coverage=0.95) 

# plt.figure(figsize=(15, 5))
# plt.plot(y_pred)
# plt.show()

### Statsforecast MSTL

In [8]:
# from statsforecast import StatsForecast
# from statsforecast.models import MSTL, HoltWinters
# from tqdm.autonotebook import tqdm

# # this makes it so that the outputs of the predict methods have the id as a column 
# # instead of as the index
# os.environ['NIXTLA_ID_AS_COL'] = '1'

In [9]:
# # = = = = = = = = = = = = = 
# # get true data
# df_energy = data_prepro.get_energy_data_today(to_date=t_wednesday_next.strftime('%Y%m%d'))

# # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# # take smaller train dataset
# start   = '2022-01-01'
# end     = df_energy['timestamp_CET'].max()
# end     = t_thursday.strftime('%Y-%m-%d')
# df_energy_small = df_energy.loc[(df_energy['timestamp_CET'] > start) &
#                                 (df_energy['timestamp_CET'] <= end)]

# print('- '*15)
# print(f"take smaller dataset ... from {start} to {end}")
# print(df_energy_small.info())

# # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# # take the required week 
# start   = t_thursday.strftime('%Y-%m-%d')
# end     = t_sunday.strftime('%Y-%m-%d')
# df_energy_true = df_energy.loc[(df_energy['timestamp_CET'] >= start) &
#                                (df_energy['timestamp_CET'] <= end)]

# print('- '*15)
# print(f"true data ... from {start} to {end}")
# print(df_energy_true.info())

### MSTL

In [10]:
# from importlib import reload
# reload(data_prepro)
# reload(model_train)
# reload(model_fcast)
# reload(model_eval)

In [11]:
# # StatsForecast needs this format
# df = df_energy_small.copy().rename(columns={'timestamp_CET': 'ds', 'gesamt': 'y'})
# df['unique_id'] = 'energy'

# # Use past 2 years data for MSTL ... take last 365 * 24 values
# df_mstl = df_energy_small.iloc[-2*365*24:].copy()
# # Use past 4 weeks data for HoltWinters
# df_hw = df_energy_small.iloc[-4*7*24:].copy()

In [12]:
# # MSTL's usage example
# mstl_model = MSTL(season_length=[24, 24*7])
# mstl_model = mstl_model.fit(df_mstl['gesamt'])

In [13]:
# # calculate how many step aheads in hours to last submission timestamp
# # to get the required number of hourly predictions
# n_steps = int((subm_timestamps[-1] - df['ds'].max()) / pd.Timedelta(hours=1)) + 5

# y_hat_dict = mstl_model.predict(h=n_steps, level=[50, 95])
# # y_hat_dict
# # turn the dictionary into a dataframe
# y_hat_df = pd.DataFrame(y_hat_dict)

# # add timestamp_CET column
# y_hat_df['timestamp_CET'] = pd.date_range(start=df['ds'].max(), periods=len(y_hat_df), freq='H')
# y_hat_df

# # rename columns
# y_hat_df = y_hat_df.rename(columns={'mean': 'q 0.500', 
#                                     'lo-50': 'q 0.250', 
#                                     'hi-50': 'q 0.750', 
#                                     'lo-95': 'q 0.025', 
#                                     'hi-95': 'q 0.975'})
# y_hat_df = y_hat_df[['timestamp_CET', 'q 0.025', 'q 0.250', 'q 0.500', 'q 0.750', 'q 0.975']]

# model_train.plot_quantile_fcast(y_hat_df, subm_timestamps, y_true=df_energy_true, title='MSTL fcast')

In [14]:
# model_eval.eval_fcast(y_hat_df, df_energy_true, subm_timestamps, quantiles)

### Holt Winters

In [15]:
# # Holt-Winters' usage example
# holt_winters_model = HoltWinters(season_length=24, error_type='A')
# holt_winters_model = holt_winters_model.fit(y=df_hw['gesamt'].values)

# # = = = = = = = = = = = = = = = = = = = = = = = = = #
# # !!! can only use 1 single seasonality parameter !!!
# # = = = = = = = = = = = = = = = = = = = = = = = = = #

In [16]:
# # calculate how many step aheads in hours to last submission timestamp
# # to get the required number of hourly predictions
# n_steps = int((subm_timestamps[-1] - df['ds'].max()) / pd.Timedelta(hours=1)) + 5

# y_hat_dict = holt_winters_model.predict(h=n_steps, level=[50, 95])
# # y_hat_dict
# # turn the dictionary into a dataframe
# y_hat_df = pd.DataFrame(y_hat_dict)

# # add timestamp_CET column
# y_hat_df['timestamp_CET'] = pd.date_range(start=df['ds'].max(), periods=len(y_hat_df), freq='H')
# y_hat_df

# # rename columns
# y_hat_df = y_hat_df.rename(columns={'mean': 'q 0.500', 
#                                     'lo-50': 'q 0.250', 
#                                     'hi-50': 'q 0.750', 
#                                     'lo-95': 'q 0.025', 
#                                     'hi-95': 'q 0.975'})
# y_hat_df = y_hat_df[['timestamp_CET', 'q 0.025', 'q 0.250', 'q 0.500', 'q 0.750', 'q 0.975']]

# model_train.plot_quantile_fcast(y_hat_df, subm_timestamps, y_true=df_energy_true, title='Holt Winters fcast')

In [17]:
# model_eval.eval_fcast(y_hat_df, df_energy_true, subm_timestamps, quantiles)

## Rolling Fcast Eval

In [23]:
from importlib import reload
reload(data_prepro)
reload(model_train)
reload(model_fcast)
reload(model_eval)

<module 'model_eval' from 'c:\\2023_11-PTSFC\\model_eval.py'>

In [19]:
# = = = = = = = = = = = = = 
# get all data
t_now = pd.Timestamp.now(tz='CET').replace(hour=0, minute=0, second=0, microsecond=0)
df_energy = data_prepro.get_energy_data_today(to_date=t_now.strftime('%Y%m%d'))

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# take smaller train dataset
start   = '2020-01-01'
end     = df_energy['timestamp_CET'].max()
df_energy_small = df_energy.loc[(df_energy['timestamp_CET'] > start) &
                                (df_energy['timestamp_CET'] <= end)]

print('- '*15)
print(f"take smaller dataset ... from {start} to {end}")
print(df_energy_small.info())

> cwd = c:\2023_11-PTSFC


> to_date is later than 2023-11-01, using recent data as well !


100%|██████████| 14/14 [00:07<00:00,  1.88it/s]


> 0 NA in df
> last valid index = 2024-01-31 18:15:00+00:00
> done and saved to 2015-01-01_2024-01-31_energy.csv
- - - - - - - - - - - - - - - 
take smaller dataset ... from 2020-01-01 to 2024-01-31 19:00:00+01:00
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35803 entries, 2020-01-01 00:00:00+00:00 to 2024-01-31 18:00:00+00:00
Freq: H
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   timestamp_CET  35803 non-null  datetime64[ns, CET]
 1   gesamt         35803 non-null  float64            
dtypes: datetime64[ns, CET](1), float64(1)
memory usage: 839.1 KB
None


In [44]:
n_fold = 5
# get past n_fold + 1 wednesdays
t_wednesday = t_now - pd.Timedelta(days=2-t_now.weekday())
past_wednesdays = [t_wednesday - pd.Timedelta(days=7*(i+1)) for i in range(n_fold)]
past_sundays = [t_wednesday + pd.Timedelta(days=4) for t_wednesday in past_wednesdays]

res_dict = {}
preds_dict = {}

for fold_idx, t_wednesday in enumerate(past_wednesdays):

    print('- '*15)
    print(f"fold {fold_idx+1}/{n_fold}")
    print('- '*15)

    wed = t_wednesday.strftime('%Y-%m-%d')
    sun = past_sundays[fold_idx].strftime('%Y-%m-%d')

    subm_timestamps = []
    for fcast in fcast_hor:
        subm_timestamps.append((t_wednesday + pd.Timedelta(days=1) + pd.Timedelta(hours=fcast)))
        
    df_train = df_energy_small.loc[df_energy_small['timestamp_CET'] <= wed]
    print(f"train from: {df_train['timestamp_CET'].min()} to {df_train['timestamp_CET'].max()}")
    df_test = df_energy_small.loc[(df_energy_small['timestamp_CET'] >= wed) & (df_energy_small['timestamp_CET'] <= sun)]
    print(f"test from:  {df_test['timestamp_CET'].min()} to {df_test['timestamp_CET'].max()}")   

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # # Use past 2 years data for MSTL ... take last 365 * 24 values
    # df_mstl = df_train.iloc[-1*365*24:].copy()
    # # Use past 4 weeks data for HoltWinters
    # df_hw = df_train.iloc[-4*7*24:].copy()

    train_horizons = [4, 3, 2, 1, 0.5, 0.25]

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # MSTL's usage example
    mstl_models = {}

    # For different train horizons (in years)
    for train_horizon in train_horizons:
        df_mstl_train = df_train.iloc[-int(train_horizon*365*24):].copy()
        mstl_model = MSTL(season_length=[24, 24*7]).fit(df_mstl_train['gesamt'])
        mstl_models[train_horizon] = mstl_model

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # Holt-Winters' usage example
    hw_models = {}

    # [TODO] for different train horizons
    for train_horizon in train_horizons:
        df_hw_train = df_train.iloc[-int(train_horizon*365*24):].copy()
        holt_winters_model = HoltWinters(season_length=24, error_type='A').fit(y=df_hw_train['gesamt'].values)
        hw_models[train_horizon] = holt_winters_model

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # Predictions
    n_steps = 100
    fcast_timestamps = pd.date_range(start=df_train['timestamp_CET'].max(), periods=n_steps, freq='H')

    models = [mstl_models, hw_models]
    model_names = ['mstl', 'holt_winters']

    for model_idx, model_dict in enumerate(models):
        for train_horizon, model in model_dict.items():
            y_hat_dict = model.predict(h=n_steps, level=[50, 95])
            y_hat_df = pd.DataFrame(y_hat_dict)

            # add timestamp_CET column
            y_hat_df['timestamp_CET'] = fcast_timestamps
            y_hat_df

            # rename columns
            y_hat_df = y_hat_df.rename(columns={'mean': 'q 0.500',
                                                'lo-50': 'q 0.250',
                                                'hi-50': 'q 0.750',
                                                'lo-95': 'q 0.025',
                                                'hi-95': 'q 0.975'})
            y_hat_df = y_hat_df[['timestamp_CET', 'q 0.025', 'q 0.250', 'q 0.500', 'q 0.750', 'q 0.975']]

            # save predictions
            preds_dict[f"{model_names[model_idx]}_{train_horizon}"] = y_hat_df

            model_train.plot_quantile_fcast(y_hat_df, subm_timestamps, y_true=df_test, 
                                            title=f"{model_names[model_idx]}-{train_horizon}y fcast")
            
            df_scores = model_eval.eval_fcast(y_hat_df, df_test, subm_timestamps, quantiles)
            # save the last row of the scores dataframe to final output
            res_dict[f"{model_names[model_idx]}_{train_horizon}"] = df_scores.iloc[-1:]

- - - - - - - - - - - - - - - 
fold 1/5
- - - - - - - - - - - - - - - 
train from: 2020-01-01 01:00:00+01:00 to 2024-01-24 00:00:00+01:00
test from:  2024-01-24 00:00:00+01:00 to 2024-01-28 00:00:00+01:00


In [42]:
# make dataframe from dictionary
df_res = pd.concat(res_dict.values())
df_res.index = res_dict.keys()
df_res

,abs error q 0.5,q-score 0.025,q-score 0.250,q-score 0.500,q-score 0.750,q-score 0.975
mstl_0.5,3619.763069,0.459895,2.504522,3.619763,3.380971,0.727475
holt_winters_0.5,7493.730251,4.842286,8.659068,7.493730,5.203441,0.797945
